Импорт библиотек

In [21]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

Загрузка данных



In [12]:
# Загрузка данных
data_path = '/content/bank_dataset_hash.csv'
data = pd.read_csv(data_path, encoding='cp1251', sep=';', low_memory=False)

Преобразование данных и обработка данных

In [13]:
# Преобразование числовых строковых данных
def replace_comma_and_convert(column):
    return pd.to_numeric(column.str.replace(',', '.'), errors='coerce')

In [14]:
# Переименование столбцов
new_columns = [
    "snapshot_dt", "agreement", "customer", "f_triggered",
    "feature1", "feature2", "feature3", "feature4",
    "feature5", "feature6", "feature7", "feature8",
    "feature9", "feature10", "feature11", "feature12",
    "feature13", "feature14", "feature15", "feature16",
    "feature17", "feature18", "feature19", "feature20",
    "feature21", "feature22", "feature23", "feature24",
    "feature25", "feature26", "feature27", "feature28",
    "feature29", "feature30", "feature31", "feature32",
    "feature33", "feature34", "feature35", "feature36",
    "feature37", "feature38", "feature39", "feature40",
    "feature41", "feature42", "feature43", "feature44",
    "feature45", "feature46", "feature47", "feature48",
    "feature49", "feature50", "feature51", "feature52",
    "feature53", "feature54", "feature55", "feature56",
    "feature57", "feature58", "feature59", "feature60",
    "feature61", "feature62"
]
data.columns = new_columns

In [15]:
# Замена значений inf, -inf, NaN, ‹Ћ†њ на пустые значения
data = data.apply(lambda col: replace_comma_and_convert(col) if col.dtype == 'object' else col)
data.replace([np.inf, -np.inf, 'inf', '-inf', 'NaN', '‹Ћ†њ'], np.nan, inplace=True)

In [16]:
# Удаление столбцов с более чем 70% пропусков
threshold = 0.7 * len(data)
columns_with_many_nans = data.columns[data.isnull().mean() > 0.7]
data = data.drop(columns=columns_with_many_nans)

In [17]:
# Удаление дубликатов по столбцу 'customer'
data = data.drop_duplicates(subset='customer', keep='first')

In [18]:
# Заполнение пустых значений медианой
data = data.fillna(data.median())

In [19]:
# Разделение данных на классы
data_class_0 = data[data['f_triggered'] == 0]
data_class_1 = data[data['f_triggered'] == 1]

# Функции для поиска выбросов
def find_outliers_iqr(df):
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = (df < lower_bound) | (df > upper_bound)
    return outliers

def find_outliers_z(df):
    z_scores = np.abs(stats.zscore(df.select_dtypes(include=[np.number])))
    outliers = (z_scores > 3)
    return pd.DataFrame(outliers, columns=df.select_dtypes(include=[np.number]).columns)

# Поиск и обработка выбросов для класса 0
outliers_iqr_0 = find_outliers_iqr(data_class_0)
outliers_z_0 = find_outliers_z(data_class_0)
outliers_combined_0 = outliers_iqr_0 | outliers_z_0
data_class_0.loc[outliers_combined_0.any(axis=1)] = np.nan  # Заменяем выбросы на NaN
data_class_0 = data_class_0.fillna(data_class_0.median())

# Поиск и обработка выбросов для класса 1
outliers_iqr_1 = find_outliers_iqr(data_class_1)
outliers_z_1 = find_outliers_z(data_class_1)
outliers_combined_1 = outliers_iqr_1 | outliers_z_1
data_class_1.loc[outliers_combined_1.any(axis=1)] = np.nan  # Заменяем выбросы на NaN
data_class_1 = data_class_1.fillna(data_class_1.median())

# Объединение данных обратно
data_processed = pd.concat([data_class_0, data_class_1])

# Проверка распределения классов после обработки выбросов
print("Class distribution after handling outliers separately:\n", data_processed['f_triggered'].value_counts())

# Сохранение итогового обработанного датасета
data_processed.to_csv('/content/processed_data_no_outliers.csv', index=False)

Class distribution after handling outliers separately:
 f_triggered
0.0    98358
1.0     2008
Name: count, dtype: int64


Обучение и оценка моделей

In [23]:
# Разделение данных на тренировочные и тестовые выборки
X = data_processed.drop(columns=['f_triggered'])
y = data_processed['f_triggered']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Применение SMOTE для увеличения числа примеров класса 1.0
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Стандартизация признаков
scaler = StandardScaler()
columns_to_scale = X_train.columns.difference(['snapshot_dt', 'agreement', 'customer'])
X_train_smote[columns_to_scale] = scaler.fit_transform(X_train_smote[columns_to_scale])
X_test[columns_to_scale] = scaler.transform(X_test[columns_to_scale])

# Функция для расчета Gini коэффициента
def gini_score(y_true, y_pred):
    roc_auc = roc_auc_score(y_true, y_pred)
    return 2 * roc_auc - 1

# Обучение и оценка моделей
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=42)
}

gini_scores = {}

for name, model in models.items():
    model.fit(X_train_smote, y_train_smote)
    y_pred = model.predict_proba(X_test)[:, 1]
    gini = gini_score(y_test, y_pred)
    gini_scores[name] = gini

# Вывод результатов
print("Gini баллы моделей:")
for name, gini in gini_scores.items():
    print(f"{name}: {gini}")

# Определение наилучшей модели
best_model_name = max(gini_scores, key=gini_scores.get)
best_model = models[best_model_name]

# Оценка важности признаков для наилучшей модели
importances = None
if best_model_name in ['Random Forest', 'Gradient Boosting']:
    importances = best_model.feature_importances_
    feature_importances = pd.Series(importances, index=X_train.columns).sort_values(ascending=False)
elif best_model_name == 'Logistic Regression':
    importances = best_model.coef_[0]
    feature_importances = pd.Series(importances, index=X_train.columns).sort_values(ascending=False)

Gini баллы моделей:
Logistic Regression: 0.819018915934514
Random Forest: 0.9959477171251623
Gradient Boosting: 0.9975858178501127


In [ ]:
print(f"Важные характеристики для лучшей модели ({best_model_name}):")
print(feature_importances)

### ***EDA***

In [ ]:
def display_columns_in_batches(data, batch_size=10, row_count=20):
    num_cols = data.shape[1]
    for start in range(0, num_cols, batch_size):
        end = start + batch_size
        print(f"Displaying columns {start + 1} to {min(end, num_cols)}:")
        display(data.iloc[:row_count, start:end])
        print("\n" + "="*80 + "\n")

display_columns_in_batches(data, batch_size=10, row_count=50)

In [ ]:
# Поиск выбросов с использованием IQR
def find_outliers_iqr(df):
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)
    IQR = Q3 - Q1
    outliers = (df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR))
    return outliers

outliers_iqr = find_outliers_iqr(data)
print("IQR-based outliers:")
print(outliers_iqr)

# Поиск выбросов с использованием Z-оценки
def find_outliers_z(df):
    z_scores = np.abs(stats.zscore(df.select_dtypes(include=[np.number])))
    outliers = (z_scores > 3)
    return pd.DataFrame(outliers, columns=df.select_dtypes(include=[np.number]).columns)

outliers_z = find_outliers_z(data)
print("Z-score-based outliers:")
print(outliers_z)

# Вывод столбцов с выбросами
outliers_columns_iqr = outliers_iqr.columns[outliers_iqr.any()].tolist()
outliers_columns_z = outliers_z.columns[outliers_z.any()].tolist()

print(f"Columns with IQR-based outliers: {outliers_columns_iqr}")
print(f"Columns with Z-score-based outliers: {outliers_columns_z}")

In [ ]:
def detect_outliers_zscore(data, threshold=3):
    outliers = pd.DataFrame(index=data.index)
    for column in data.select_dtypes(include=[np.number]).columns:
        z_scores = np.abs(stats.zscore(data[column].dropna()))
        is_outlier = z_scores > threshold
        outliers[column] = is_outlier
    return outliers

outliers_zscore = detect_outliers_zscore(data)

outliers_zscore.sum()

In [ ]:
# Функция для визуализации выбросов
def plot_outliers(df, outliers):
    fig, axs = plt.subplots(nrows=len(df.columns), figsize=(10, 50))
    for i, col in enumerate(df.columns):
        sns.boxplot(x=df[col], ax=axs[i])
        axs[i].set_title(f'Boxplot of {col}')
        outliers_col = df[outliers[col]]
        sns.scatterplot(x=outliers_col[col], y=[0]*len(outliers_col), color='red', ax=axs[i])
    plt.tight_layout()
    plt.show()

# Вывод выбросов
plot_outliers(data, outliers_iqr | outliers_z)

In [ ]:
# Проверка результата
print(data.info())
print(data.head())

In [ ]:
# Гистограммы числовых переменных
numeric_cols = data.select_dtypes(include=['float64', 'int64']).columns

print("\nГистограммы числовых переменных:")
data[numeric_cols].hist(bins=30, figsize=(15, 10))
plt.tight_layout()
plt.show()

# Графики распределения для категориальных переменных
categorical_cols = data.select_dtypes(include=['object']).columns

print("\nГрафики распределения для категориальных переменных:")
for col in categorical_cols:
    plt.figure(figsize=(10, 4))
    sns.countplot(data[col])
    plt.title(f'Распределение переменной {col}')
    plt.show()

# Тепловая карта корреляций
print("\nТепловая карта корреляций:")
plt.figure(figsize=(12, 8))
correlation_matrix = data.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Тепловая карта корреляций')
plt.show()

# Выявление выбросов с помощью ящичных диаграмм
print("\nЯщичные диаграммы для выявления выбросов:")
for col in numeric_cols:
    if data[col].nunique() > 1:  # Проверка, что в колонке больше одного уникального значения
        plt.figure(figsize=(10, 4))
        sns.boxplot(data[col])
        plt.title(f'Ящичная диаграмма переменной {col}')
        plt.show()

# Анализ пропусков
print("\nВизуализация пропусков:")
plt.figure(figsize=(12, 8))
sns.heatmap(data.isnull(), cbar=False, cmap='viridis')
plt.title('Визуализация пропусков')
plt.show()